In [1]:
!pip install tensorflowjs

     |████████████████████████████████| 61kB 2.3MB/s 
     |████████████████████████████████| 104.6MB 47kB/s 
     |████████████████████████████████| 256kB 40.8MB/s 
     |████████████████████████████████| 921kB 45.7MB/s 
     |████████████████████████████████| 3.9MB 41.5MB/s 
     |████████████████████████████████| 450kB 41.9MB/s 
  Created wheel for PyInquirer: filename=PyInquirer-1.0.3-cp36-none-any.whl size=32851 sha256=251fe40a511a97e1069c1254904470527a29d12d103148eb47aef39e7253aff1
  Stored in directory: /root/.cache/pip/wheels/52/6c/b1/3e4b0e8daf42a92883c7641c0ea8ffb62e0490ebed2faa55ad
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=bce8d07d77aab9f4b5b222b25776509ac344feced01854372fc2ddb9a41faa22
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built PyInquirer gast
ERROR: tensorflow 2.2.0rc2 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR:

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals
import os

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x  #gpu
except Exception:
  pass
  
import cProfile
import numpy as np
import pandas as pd

from keras.models import Sequential
from keras import applications

import tensorflow as tf
from tensorflow import feature_column
from tensorflow.keras import layers
from tensorflow.keras import backend as k
from tensorflow.keras import Input

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import tensorflowjs as tfjs

import json

print(tf.__version__)
tf.executing_eagerly()

SEED=10946
os.environ['PYTHONHASHSEED']=str(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x  #gpu`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.


Using TensorFlow backend.


2.1.0


In [0]:
batch_size = 20
training_epochs = 5

In [6]:
with open('/content/trainingData.json') as f:
  train_file = json.load(f)
train_a = np.array(train_file["data"])

with open('/content/labelData.json') as f:
  label_file = json.load(f)
label_a = np.array(label_file["data"])

with open('/content/inputData.json') as f:
  input_file = json.load(f)
input_a = np.array(input_file["data"])

print("Training shape = ", train_a.shape)
print("Label shape = ", label_a.shape)  

Training shape =  (5032, 199)
Label shape =  (5032, 8)


In [7]:
train_X, test_X, train_y, test_y = train_test_split(train_a, label_a, test_size=0.2)
print(len(train_X), 'train examples')
print(len(test_X), 'test examples')

train_ds = tf.data.Dataset.from_tensor_slices((train_X, train_y)).batch(batch_size)
test_ds = tf.data.Dataset.from_tensor_slices((test_X, test_y)).batch(batch_size)

4025 train examples
1007 test examples


In [8]:
model = tf.keras.Sequential([
  layers.BatchNormalization(input_shape=(199,)),  
  layers.Dense(128, activation='relu'),
  layers.Dropout(0.25),  
  layers.Dense(8, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_ds, validation_data=test_ds, epochs=training_epochs)

model.summary()

# Get stats from the history object
acc = history.history['accuracy'][training_epochs - 1]
val_acc = history.history['val_accuracy'][training_epochs - 1]


Train for 202 steps, validate for 51 steps
Epoch 1/5
202/202 [==============================] - 1s 6ms/step - loss: 0.2725 - accuracy: 0.9111 - val_loss: 0.1946 - val_accuracy: 0.9496
Epoch 2/5
202/202 [==============================] - 1s 3ms/step - loss: 0.1796 - accuracy: 0.9468 - val_loss: 0.1764 - val_accuracy: 0.9494
Epoch 3/5
202/202 [==============================] - 1s 3ms/step - loss: 0.1617 - accuracy: 0.9485 - val_loss: 0.1750 - val_accuracy: 0.9489
Epoch 4/5
202/202 [==============================] - 1s 3ms/step - loss: 0.1501 - accuracy: 0.9493 - val_loss: 0.1746 - val_accuracy: 0.9479
Epoch 5/5
202/202 [==============================] - 1s 3ms/step - loss: 0.1410 - accuracy: 0.9505 - val_loss: 0.1760 - val_accuracy: 0.9480
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 199)               796       
__________________________________

In [0]:
input_ds = tf.data.Dataset.from_tensor_slices(input_a).batch(batch_size)
predictions = model.predict(input_ds)

with open('/content/predictions.csv', 'w') as f:
  for i in range(0, predictions.shape[0]):    
    f.write(str(predictions[i][0]) + ',' + str(predictions[i][1]) + ',' + str(predictions[i][2]) + ',' + str(predictions[i][3]) + ',' + str(predictions[i][4])+ ',' + str(predictions[i][5])+ ',' + str(predictions[i][6])+ ',' + str(predictions[i][7])+ '\n')
